In [1]:
# 지금부터 서울대학교 수강신청 챗봇을 구현할 것

In [39]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory

In [25]:
# 1. pdf 불러오기
loader = PyPDFLoader('../../assets/pdf/ch02_review/광운대_졸업하기.pdf')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
chunks = text_splitter.split_documents(documents)
print('청크 수 : ', len(chunks))

청크 수 :  79


In [27]:
# 임베딩 벡터로 저장하기
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory='../../assets/embedding/ch02_review/')

In [32]:
# 문서 수 확인
print('문서의 수 : ', chroma_db._collection.count())
# 관련된 내용 3개 찾기
retriever = chroma_db.as_retriever(search_kwargs={'k':3})

문서의 수 :  79


In [35]:
template = """
당신은 광운대학교 학생들이 졸업을 할 수 있도록 도와주는 매니저입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
Context: {context}
"""
# 프롬프트 작성
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', template),
        ('placeholder', '{chat_history}'),
        ('user', '{question}')
    ]
)
model = ChatOpenAI(model='gpt-4o', temperature=0.0)

In [34]:
# 중요한 정보를 이어붙이는 함수 선언
def concatDocs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [38]:
chain = (
    RunnablePassthrough.assign(
        context=lambda x: concatDocs(retriever.invoke(x['question']))
    )|
    prompt|
    model|
    StrOutputParser()
)

In [40]:
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history')

In [45]:
def chatbot():
    session_id = "user_session"
    print('안녕하세요. 저는 광운대학교 챗봇입니다. 광운대학교 졸업 요건에 대해서 궁금한 점을 작성하세요! (종료를 위해서는 quit를 입력해주세요)')
    while True:
        question = input('사용자 : ')
        if question.lower()=='quit':
            break
        answer = chain_with_memory.invoke(
            {'question':question},
            {'configurable':{'session_id':session_id}}
        )
        print('챗봇 : ', answer)

In [46]:
chatbot()

안녕하세요. 저는 광운대학교 챗봇입니다. 광운대학교 졸업 요건에 대해서 궁금한 점을 작성하세요! (종료를 위해서는 quit를 입력해주세요)


사용자 :  안녕하세요?


챗봇 :  안녕하세요! 무엇을 도와드릴까요? 졸업 요건이나 교과목 관련해서 궁금한 점이 있으시면 언제든지 질문해 주세요.


사용자 :  졸업하기 위해서는 몇학점을 채워야하나요?


챗봇 :  질문해주신 졸업 이수 학점에 대한 정보는 입학 연도와 전형에 따라 다릅니다. 예를 들어, 2016학년도에 신입학한 학생의 경우 공학계열 학과는 30학점을 이수해야 합니다. 만학도 전형, 외국인 전형, 특성화고를 졸업한 재직자 전형, 체육특기자, 편입생의 경우 교양 필수 이수가 면제되며, 교양 학점 30학점을 이수해야 합니다.

입학 연도나 전형에 따라 구체적인 졸업 이수 학점이 다를 수 있으니, 본인의 상황에 맞는 정보를 확인해 보시기 바랍니다. 추가적인 질문이 있으면 언제든지 말씀해 주세요!


사용자 :  2022학년도 컴퓨터공학과로 입학했습니다. 자세한 이수학점 알려주세요


챗봇 :  2022학년도에 컴퓨터공학과로 입학하신 경우, 졸업을 위해 필요한 이수 학점은 다음과 같습니다:

- **교양 학점**: 22학점
- **주전공 학점(필수 포함)**: 60학점

총 졸업 이수 학점은 133학점입니다. 

이 외에도 심화전공, 복수전공, 부전공을 선택하실 경우 추가적인 학점 이수가 필요할 수 있습니다. 본인의 학업 계획에 따라 필요한 학점을 잘 확인하시고 이수 계획을 세우시기 바랍니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!


사용자 :  교양 22학점에 전공 60학점을 더하면 82학점 아닌가요?


챗봇 :  죄송합니다. 제가 잘못된 정보를 드렸습니다. 2022학년도에 컴퓨터공학과로 입학하신 경우, 졸업을 위해 필요한 총 이수 학점은 133학점입니다. 

교양과 전공 학점 외에도 일반 선택 과목 등을 포함하여 총 133학점을 이수해야 합니다. 교양과 전공 학점 외에 나머지 학점은 일반 선택 과목 등을 통해 채우실 수 있습니다. 

정확한 학점 이수 계획을 위해 학과 사무실이나 학사 관련 부서에 문의하시는 것도 좋은 방법입니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!


사용자 :  quit
